<p style="text-align: center;">
UNIVERSIDADE FEDERAL DE SANTA CATARINA.<br>
Tainá da Silva.<br>
Orientador: Luiz Rafael dos Santos. <br>
2021.</p><br>


<h1 style="text-align: center;"> Métodos iterativos para solução de sistemas lineares: aceleração usando reflexões circuncentradas</h1>

Neste trabalho estudamos a aplicação do Método de reflexões circuncentradas (CRM), recentemente desenvolvido em [2-5], na aceleração de métodos iterativos que se baseiam em projeções ortogonais para   encontrar uma solução de um sistema de equações lineares dado por
\begin{equation}\label{eq:SL}
    Ax = b
\end{equation}
em que $A\in\mathbb{R}^{m \times n}$ é uma matriz $m\geq n$, esparsa e potencialmente de larga-escala, e ainda $x\in\mathbb{R}^{n}$ e $b\in\mathbb{R}^{m}$.


Para tal, consideramos métodos que utilizam projeções sobre o hiperplano definido por cada linha do sistema linear dado em $Ax=b$. Tais projeções (e reflexões) têm um baixo custo computacional e, ao mesmo tempo, em geral são facilmente paralelizáveis. Alguns  algoritmos comumente usados neste caso incluem o algoritmo sequencial de Kaczmarz (KACZ) bem como vários algoritmos paralelos em bloco, dentre eles, o método de Cimmino (CIM) e o método de média de componentes (CAV) [6]. 


## Kaczmarz

<p>A solução do sitema linear não-singular </p>
<p>
$$
\begin{bmatrix}
    a_{11} & a_{12}\\
    a_{21} & a_{22}     
\end{bmatrix}
\cdot 
\begin{bmatrix}
x_{1}\\
x_{2}
\end{bmatrix}
=
\begin{bmatrix}
b_{1}\\
b_{2}
\end{bmatrix}
$$
</p>
<p> é a intersecção de dois hiperplanos (neste caso duas retas) definidos por </p>

$$
\mathcal{H}_{1} = \{(x_{1}, x_{2}) \mid a_{11} \cdot x_{1} + a_{12} \cdot x_{2} = b_{1}\} \hbox{ e } \mathcal{H}_{2} = \{(x_{1}, x_{2}) \mid a_{21} \cdot x_{1} + a_{22} \cdot x_{2} = b_{2}\}.
$$

<p> É visualmente evidente que, começando com um ponto arbitrário $P_{0}$ e realizando uma squência de projeções ortogonais alternadas em $\mathcal{H}_{1}$ e $\mathcal{H}_{2}$ como representado na figura abaixo, a sequência resultante de projeções $\{p_{1}, p_{2}, p_{3}, p_{4}, \ldots \}$ converge para $\mathcal{H}_{1}\cap \mathcal{H}_{2}$, que é a solução de $Ax=b$.</p>
     

![Imagem](https://i.ibb.co/BBRs4cs/kaczmarzs.jpg)

In [1]:
using LinearAlgebra, Random, BenchmarkTools
Random.seed!(3)

n=111
r=200
A = randn(n,r)
b = randn(n)

#Primeiro precisamos fazer uma função que normalize as linhas da matriz A

function normalize_matriz(A,b)
    n,r = size(A)
   for i=1:n
        β = norm(A[i,:])
        A[i,:] = A[i,:]/β
        b[i] = b[i]/β
    end
    return A,b
end

normalize_matriz (generic function with 1 method)

In [2]:
function proj(p₀, u, β) 
    
   return  p₀ - ((dot(u, p₀)- β)/dot(u, u))*u
end

proj (generic function with 1 method)

In [3]:
function Kaczmarz(A, b; itmax=10000, ε= 1e-3)
   n,r = size(A) 
    pₖ = ones(r)
    k = 1 
    A,b = normalize_matriz(A,b)
    tol = norm(b- A*pₖ)
    while k <= itmax && tol> ε   
    for i=1:n
         pₖ= proj(pₖ, A[i,:], b[i])
    end
        tol = norm(b- A*pₖ)
        k += 1
    end
    
    return pₖ, tol, k
end

Kaczmarz (generic function with 1 method)

In [4]:
Kaczmarz(A, b, ε=1e-6)

([0.07236108286868863, -0.03898190532879666, 0.12475449945437528, -0.2781786847646876, 1.1269185531232808, 0.7920657523442495, 0.33661464290211324, 0.1310469531790241, 0.07387319011758799, 0.7301552060638411  …  -0.19383310177276486, 0.6180487196412655, 0.8183045597179235, 0.3170114229352824, 1.2999397012828742, 0.5536947415729248, 0.3724958520258866, -0.26052358659440095, -0.34223083956496914, 1.2246465445148953], 8.864136205630041e-7, 77)

## Cimmino

<p style="text-align: justify; text-justify: inter-word"> O matemático italiano Gianfranco Cimmino usou a seguinte observação elementar para construir um algoritmo iterativo para resolver sistemas lineares. Para um sistema $2 \times 2$ tem $Ax = b$, sejam $\mathcal{H}_{1}$ e $\mathcal{H}_{2}$ as duas retas (hiperplanos) definidas pelas duas equações. Para um palpite arbitrário $r_{0}$, seja $r_{1}$ o reflexão de $r_{0}$ sobre a linha $\mathcal{H}_{1}$, e seja $r_{2}$ o reflexão de $r_{0}$ sobre a linha $\mathcal{H}_{2}$. Conforme ilustrado na abaixo, os três pontos $r_{0}$, $r_{1}$ e $r_{2}$ encontram-se em um círculo cujo centro é $\mathcal{H}_{1} \cap \mathcal{H}_{2}$ (a solução do sistema). </p>

![Imagem](https://i.ibb.co/CMrVJfc/cimmino.jpg)

<p style="text-align: justify; text-justify: inter-word"> O valor médio $m = \frac{(r_{1} + r_{2})}{2}$ está estritamente dentro do círculo, então $m$ é uma melhor aproximação da solução do que $r_{0}$. É visualmente evidente que iteração produz uma sequência que converge para a solução de $Ax = b$. Provaremos isso em geral usando o esquema a seguir. </p>

In [5]:
function reflexao(p₀, u, β) 
    
   return  2*proj(p₀, u, β) - p₀
end

reflexao (generic function with 1 method)

In [6]:
function Cimmino(A, b; itmax=10000, ε= 1e-3)
    n,r = size(A) 
    xₖ = ones(r)
    k = 1 
    A,b = normalize_matriz(A,b)
    tol = norm(b- A*xₖ)
    
    while k <= itmax && tol> ε 
        rₖ = zeros(r)
        for i=1:n
         rₖ += reflexao(xₖ, A[i,:], b[i])
        end
        xₖ= rₖ/n
        tol = norm(b- A*xₖ)
        k += 1
    end
    return xₖ, tol, k
end

Cimmino (generic function with 1 method)

In [7]:
Cimmino(A, b, ε=1e-3)

([0.07278968994792681, -0.039385854618292825, 0.12469450836399668, -0.2777793274599455, 1.1267132363887353, 0.7920233410340194, 0.33666095663170564, 0.13123579780334507, 0.07395673091775246, 0.7303097408385365  …  -0.19422691079925913, 0.6176740495933697, 0.8178942898139873, 0.3170120335404509, 1.2999353332230903, 0.5536919659364684, 0.37292251763582374, -0.26033103992332274, -0.34224436658768964, 1.22463219059066], 0.0009990975689428418, 3682)

## CAV

In [9]:
function Esparsidade(A::SparseMatrixCSC)
	S = Int64[]
	for col in eachcol(A)
		push!(S, nnz(col))
	end
	return S
end

LoadError: UndefVarError: SparseMatrixCSC not defined

In [10]:
function norma_S_matriz(A₀, b₀; S::Vector=[]) #Caso não seja fornecida a E, cacula norma Euclidiana
    n, r = size(A₀)
	if isempty(S) 
		S = ones(r)
	end   
	for i=1:n
		βᵢ = A₀[i,:]'*S[i]*A₀[i,:]
        A₀[i,:] /=βᵢ
        b₀[i] /= βᵢ
       end
       return A₀, b₀
    end

norma_S_matriz (generic function with 1 method)

In [11]:
function CAV_sparse(A::SparseMatrixCSC, b::Vector; itmax::Int=10_000, ε::Float64= 1e-3)
	n,r = size(A)
	xₖ = ones(r)
	k = 0
	E = Esparsidade(A)
	A,b = norma_S_matriz(A, b; S=E) 
	tol = norm(b - A*xₖ)
 		while k <=itmax && tol> ε 
			xₖ = reflexao_simultanea(xₖ, A, b, n, r)
			tol = norm(b- A*xₖ)
			k += 1
		end
	return xₖ, tol, k
end

LoadError: UndefVarError: SparseMatrixCSC not defined

In [12]:
spa = sprand(100, 100, 0.05)
vet_spa = rand(100)

LoadError: UndefVarError: sprand not defined

In [13]:
CAV_sparse(spa, vet_spa, itmax=10_000)

LoadError: UndefVarError: spa not defined

In [14]:
Cimmino(spa, vet_spa, itmax=10_000)

LoadError: UndefVarError: spa not defined

## CRM

O esquema do CRM pode ser entendido da seguinte forma: considere apenas o problema de melhor aproximação de um ponto $x\in\mathbb{R}^n$ para a interseção de dois conjuntos $K_{1},K_{2}\subset\mathbb{R}^{n}$, para os quais é possível calcular uma projeção. O próximo iterado do método é o circuncentro do triangulo de vértices $x$, $y:= R_{K_{1}}(x)$ e $z:= R_{K_{2}}R_{K_{1}}(x)$, denotado por 
\begin{equation}C_T(x):=circum(x,y,z).
\end{equation}
em que $R_X=2P_X-Id$ é a reflexão em relação a um conjunto $X$ e  $Id$ é o operador identidade. Note que $P_X(x)$ é tão somente o ponto médio do segmento $[x,R_X(x)]$.

Por circuncentro queremos dizer que  $C_T(x)$ é equidistante a $x$, $y$ e $z$ e está alocado no subespaço afim definido por estes três vértices. Dados dois  conjuntos $K,U$  convexos e fechados, com $U$ afim, para todo $x\in \mathbb{R}^{n}$,  $C_T(x)$ existe, é unicamente determinado e tem uma fórmula fechada para seu cálculo. Existência e unicidade são óbvias se a cardinalidade do conjunto $\{x,y,z\}$ é $1$ ou  $2$. Com efeito, se acontecer de  $x=y=z$, já teremos $C_T(x)=x\in K\cap U$  --- a recíproca também é verdadeira, isto é, se $C_T(x)=x$, então $x=y=z$. Isto significa que o conjunto dos pontos fixos  $Fix C_T$ do operador  (não-linear)   $C_T$ é igual à $U\cap V$. Se a cardinalidade de $\{x,y,z\}$ for $2$ então $C_T(x)$ é o ponto médio entre os dois pontos distintos. Se $x$, $y$ e $z$  são distintos, tanto a existência quanto unicidade seguem de argumentos de geometria elementar. Assim, deve ser preocupação apenas o caso em que   $x$, $y$ e $z$  são distintos porém colineares. Entretanto,  isto não pode acontecer no caso em questão, pois reflexões em convexos são operações que preservam norma. Mais que isso,  a distância entre $x$, $y$ e $z$ à $K\cap U$ é exatamente a mesma. Portanto, a equidistância que pedimos em  \begin{equation}C_T(x):=circum(x,y,z).
\end{equation} torna-se uma condição necessária para a solução de $\bar x\in S\text{ tal que } \|\bar x-x^{0}\|=\min_{s\in S} \|s-x^{0}\|.$ Uma representação esquemática desta ideia pode ser encontrada  na~\Cref{figure2}.


![Imagem](https://i.ibb.co/Wtb4VJF/DRC-Ex-Sphere2.jpg)

![Imagem](https://i.ibb.co/LNVwDdj/DRC-Ex-Plane4.jpg)

[1]  Baushke H. H., Ouyang, H. and Wang, X.On Circumcenters of Finite Sets in Hilbert Spaces.Linear Nonlinear Anal. 4(2):271–295, 2018.

[2]  Behling,  R.,  Bello-Cruz,  J.  Y.  and  Santos,  L.-R.Circumcentering  the  Douglas–Rachfordmethod. Numer. Algorithms. 78(3):59–776, 2018. DOI: 10.1007/s11075-017-0399-5.

[3]  Behling,    R.,    Bello-Cruz,    J.   Y.   and   Santos,    L.-R.On   the   linear   convergence   ofthe    circumcentered-reflection    method.   Oper.   Res.   Lett.   46(2):159–162,2018.   DOI:10.1016/j.orl.2017.11.018.

[4]  Behling,  R.,  Bello-Cruz,  J.  Y.  and  Santos,  L.-R.The  Block-wise  Circumcentered-ReflectionMethod. Comput. Optim. Appl. 76(3):675–699, 2020. DOI: 10.1007/s10589-019-00155-0.

[5]  Behling,   R.,   Bello-Cruz,   J.   Y.   and   Santos,   L.-R.On   the   Circumcentered-ReflectionMethod  for  the  Convex  Feasibility  Problem.  Numer.  Algorithms.  86:1475–1494,  2021.  DOI:10.1007/s11075-020-00941-6.[6]  Elble,   J.  M.,   Sahinidis,   N.  V.  and  Vouzis,   P.GPU   computing   with   Kaczmarz’s   andother  iterative  algorithms  for  linear  systems. Parallel Comput. 36(5–6):215–231, 2010. DOI:10.1016/j.parco.2009.12.003.